> # Notebook para el EDA

In [1]:
# Librerias
import pandas as pd
import yaml
import numpy as np

# Cargando Yaml
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [17]:
df = pd.read_excel(r"../Data/Base de datos MITOS.xlsx")

>> ## Exploración y Valores Nulos

>>>### Parte I

Dado a que la investigación fue realizada dividiendo a los individuos en dos grupos, para aplicar 2 tratamientos distintos en 2 tiempos, revisaremos las siguientes columnas:

- visita
- grupo
- periodo
- tiempo
- tratamiento

Todos los individuos deberían de tener 4 visitas y un grupo, periodo y tiempo asignado. Las llamaremos **columnas identificadoras** pues únicamente sirven para identificar a los individuos así como a su tiempo/fase en el experimento.

In [3]:
# Contando registros por persona
df['visita'].value_counts().reset_index()

,visita,count
0,1,47
1,2,47
2,3,47
3,4,47


In [4]:
col_identificadoras = ['grupo', 'periodo', 'tiempo', 'tratamiento']

df[col_identificadoras].isna().sum()

grupo          6
periodo        8
tiempo         8
tratamiento    6
dtype: int64

In [5]:
df[df[col_identificadoras].isna().any(axis=1)]['id'].unique()

array([42, 46])

Podemos observar que los IDs 42 y 46 tienen valores nulos en las columnas identificadoras.

Dada la consulta con los compañeros de España, se decidió darles el siguiente tratamiento: **excluir a los individuos**

>>>### Parte II

Debido a que la base de datos se encuentra en un formato wide, lo transformaremos a uno deep para identificar si algún individuo no tiene alguna visita.

In [6]:
df_deep = df.copy()

# Eliminando columnas identificadoras para no confundirnos
df_deep.drop(columns=col_identificadoras,inplace=True)

# Transformando en formato deep
df_deep = pd.melt(df_deep, id_vars=['id', 'visita'], value_name='valor')

# Agrupando para encontrar nulos individuos sin ningun valor en una visita
df_visita_nula = df_deep.groupby(['id', 'visita'])['valor'].apply(lambda x: x.isnull().mean()).reset_index()

In [7]:
# Filtrando a los IDs que tengan visitas totalmente vacías
df_visita_nula[df_visita_nula.valor == 1]

,id,visita,valor
165,42,2,1.0
166,42,3,1.0
167,42,4,1.0
181,46,2,1.0
182,46,3,1.0
183,46,4,1.0


Podemos identificar a los individuos 8 y 26 que no tienen ningún valor en la cuarta visita.

Dada la consulta con los compañeros de España, se decidió darles el siguiente tratamiento: **excluir a los individuos**

In [8]:
# Excluyendo a los individuos que los compañeros nos indican, por ausencia total de información relevante.
df = df[~df['id'].isin([42, 46, 8, 26, 30])]

# Eliminando df_deep
del df_deep, df_visita_nula

In [9]:
nulls_count = df.isna().sum().reset_index()

nulls_count.columns = ['Variable','Null_Count']

In [10]:
nulls_count[nulls_count['Null_Count'] > 0]

,Variable,Null_Count
6,frec_alcohol_audit,128
7,puntaje_audit,128
8,Palabras_moca,126
9,memoria_moca,126
10,puntaje_moca,126
...,...,...
647,delta_d2_con_pd,84
648,delta_d2_vt_pd,84
649,delta_fn_nombre,84
650,delta_fn_score,84


>> ## Tratamiento de Variables y Valores Nulos

Tenemos **309** variables con valores nulos.

Dada nuestra conversación con los compañeros de Barcelona, nos juntamos para definir que tratamiento darles a los valores nulos de cada variable.

Aprovechando que colaboramos con expertos en el tema, pudimos tomar una decisión precisa para cada una de ellas.
Se optó por eliminar ciertas variables que no estaban muy completas o que ellos no consideraron relevantes para el estudio.

Algunas otras se les dará tratamiento especial y en otras llenaremos los valores nulos con la técnica que se mencionará a continuación.

>>> ### Tratamiento **Especial**

Estas variables, recibimos instrucciones especiales para tratar con ellas o con los valores nulos, a continuación el detalle por variable.

In [12]:
# ---------- EDAD ----------
# Dado que esta columna solamente tiene una observación por Id, la rellenaremos con la fecha de nacimiento del mismo individuo.
df['f_nac'] = df['f_nac'].ffill()

# Posteriormente, se nos indicó calcular la edad al momento de la visita. Se calculará a 2 decimales.
df['edad'] = ((pd.to_datetime(df['fecha_visita']) - pd.to_datetime(df['f_nac'])).dt.days / 365).round(2)

# Eliminando Fecha de Nacimiento
del df['f_nac']

C:\Users\mardo\AppData\Local\Temp\ipykernel_1728\3483543421.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['f_nac'] = df['f_nac'].ffill()


In [ ]:
df = df.copy()
# ---------- MEDICAMENTOS ----------
# Estas variables, tienen de manera escrita que medicamentos usa, se contará cuántos usa y se transformará a nulos para eliminar el resto.
med_columns = config['Tratamiento_Variables']['Especial'][1:7]

# Cuenta solo si NO es nulo Y ademas NO es una cadena vacía
df['total_meds'] = (df[med_columns].notna()).sum(axis=1)

# Eliminando las columnas med
df = df.drop(columns=med_columns)
del med_columns

In [26]:
min_f_visita = min(pd.to_datetime(df['fecha_visita']))
max_f_visita = max(pd.to_datetime(df['fecha_visita']))

# Para las variables de menstruación, se eliminarán las fechas y creará una variable categórica
menst_columns = config['Tratamiento_Variables']['Especial'][7:9]

# Convirtiendo a fechas
df[menst_columns] = df[menst_columns].apply(pd.to_datetime, errors='coerce')
# Validando fechas de menstruación
condicion_c1 = df[menst_columns[0]].between(min_f_visita, max_f_visita)
condicion_c2 = df[menst_columns[1]].between(min_f_visita, max_f_visita)

# Creando flag de menstruación
df['menst_flag'] = (condicion_c1 | condicion_c2).astype(int)

# Validando una vez más, que en hombres sea 0 la columna
df.loc[df['sexo'] == 1, 'menst_flag'] = 0

# Eliminando las variables que no se utilizarán
df.drop(columns=menst_columns)
del min_f_visita, max_f_visita, menst_columns

In [ ]:
# Variable numero de depocisiones
# Se estandarizará a valores: 'Una Por Día', '< Una Por Día', 'Dos + Por Día'

# Investigando valores únicos
df.num_depo.unique()

array([nan, '1/día', '1-2/dia', '6/semana', '5/semana', 'cada 2 días',
       '2/día', 'Diariamente', '4/semana', '3/día', '1-2/día',
       'diariamente', '2-3/día', 'Diariamente, hasta 2/día', '5/setmana',
       '4/setmana', '2/dia', 'Cada 2 días', '6/setmana', '1/dia',
       '4veces/semana', '4 veces/semana', '3 veces/semana',
       'Cada 2-3 días', '1-2 veces/día',
       'Última semana a diario. Anteriormente cada 4 días',
       'Cada 4-5 días', '1 vez/semana', '2 veces/semana', '2 veces/día',
       '1/día, a veces 2', '2-3 veces/día'], dtype=object)

In [ ]:
# Creando diccionario con valores únicos
depo_dict = {
    '1/día': 'Una Por Día', 
    '1-2/dia': 'Dos + Por Día', 
    '6/semana': '< Una Por Día', 
    '5/semana': '< Una Por Día', 
    'cada 2 días': '< Una Por Día',
    '2/día': 'Dos + Por Día', 
    'Diariamente': 'Una Por Día', 
    '4/semana': '< Una Por Día', 
    '3/día': 'Dos + Por Día', 
    '1-2/día': 'Dos + Por Día', 
    'diariamente': 'Una Por Día', 
    '2-3/día': 'Dos + Por Día', 
    'Diariamente, hasta 2/día': 'Dos + Por Día', 
    '5/setmana': '< Una Por Día', 
    '4/setmana': '< Una Por Día', 
    '2/dia': 'Dos + Por Día', 
    'Cada 2 días': '< Una Por Día', 
    '6/setmana': '< Una Por Día', 
    '1/dia': 'Una Por Día', 
    '4veces/semana': '< Una Por Día', 
    '4 veces/semana': '< Una Por Día', 
    '3 veces/semana': '< Una Por Día', 
    'Cada 2-3 días': '< Una Por Día', 
    '1-2 veces/día': 'Dos + Por Día', 
    'Última semana a diario. Anteriormente cada 4 días': 'Una Por Día', 
    'Cada 4-5 días': '< Una Por Día', 
    '1 vez/semana': '< Una Por Día', 
    '2 veces/semana': '< Una Por Día', 
    '2 veces/día': 'Dos + Por Día', 
    '1/día, a veces 2': 'Dos + Por Día', 
    '2-3 veces/día': 'Dos + Por Día'
}

df['num_depo'] = df.num_depo.map(depo_dict)

Función para rellenar variables según lo estipulado por los compañeros de barcelona.

Se tomará el valor de la visita anterior correspondiente a cada ID. Esto hace sentido dado que en mediciones quimicas, los valores no suelen cambiar demasiado.
Además, que algunos de los nulos, se deben a que algunos estudios no se realizan si no ha pasado x numero de días, razón por la que no se realizó la medición en la visita siguiente.

In [ ]:
import pandas as pd
import numpy as np

def rellenar_nulos_por_id(df, lista_columnas):
    """
    Rellena nulos priorizando el valor de arriba (ffill) y luego el de abajo (bfill),
    estrictamente dentro del mismo grupo de ID.
    """
    # Creamos una copia para no modificar el dataframe original directamente
    df_resultado = df.copy()

    # Agrupamos por ID para aislar los datos
    grupo = df_resultado.groupby('ID')

    for col in lista_columnas:
        # transform mantiene el índice original, permitiendo asignar la columna de vuelta
        # 1. ffill() -> Toma el de arriba (Prioridad)
        # 2. bfill() -> Toma el de abajo (Si el de arriba era nulo o no existía)
        df_resultado[col] = grupo[col].transform(lambda x: x.ffill().bfill())

    return df_resultado

>> ## Estadísticas de Variables

Definiremos los grupos de variables según lo establecido por los estudiantes del PHD en Nutrición.

- Demografía/estilo de vida (23 variables)
- Antropometría y signos vitales (38)
- Laboratorio clínico (41)
- Uroanálisis (24)
- Actigrafía y sueño objetivo (17)
- Cuestionarios de sueño PGH/ISI (40)
- Salud mental HADS (3)
- Calidad de vida WHOQOL‑BREF (38)
- Actividad física IPAQ (9)
- Cognición —MoCA, D2, WCST y funciones nominales— (31)
- Frecuencia alimentaria (FFQ) (181 ítems, incluyendo preparaciones de alimentos)
- Consumo/preparación específica de tomate (15)
- Carotenoides/licopeno (11)
- Marcadores neurotróficos y de envejecimiento (5)

En conjunto, el contenido permite relacionar **exposición dietética (tomate y licopeno)** con **biomarcadores y resultados cognitivos**.
